In [10]:
import re
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer



def retrieve_docs_and_clean():
  df = pd.read_csv('/Users/ammarlakdawala/Documents/phoenixbio/Phoenix Bioinformatics/IF Code/FinalDatasets/phyb_100.csv')
  #df = df['abstract']
  # Clean Paragraphs
  col_list =  list(df["abstract"])

  documents_clean = []
  for d in col_list:
    if isinstance(d, str):
        document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        document_test = re.sub(r'@\w+', '', document_test)
        document_test = document_test.lower()
        document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        document_test = re.sub(r'[0-9]', '', document_test)
        document_test = re.sub(r'\s{2,}', ' ', document_test)
        documents_clean.append(document_test)

  return documents_clean

In [11]:
docs = retrieve_docs_and_clean()

# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)



In [12]:
# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
print(df.head())
print(df.shape)

               0    1    2    3    4    5    6    7    8    9    ...  969  \
aa             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
aaa            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
aaaagggaaggga  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
aaaagggaaggta  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
aac            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

               970  971  972  973  974  975  976  977  978  
aa             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
aaa            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
aaaagggaaggga  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
aaaagggaaggta  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
aac            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 979 columns]
(12218, 979)


In [13]:
docs = retrieve_docs_and_clean()
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
df.head()

,0,1,2,3,4,5,6,7,8,9,...,969,970,971,972,973,974,975,976,977,978
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaagggaaggga,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaagggaaggta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def get_similar_articles(q, df):
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in df.columns:
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similarity Value:", v)
      print(docs[k])
      print()


q1 = 'phyB'
get_similar_articles(q1, df)
print('-'*100)

query: phyB
The following are articles with the highest cosine similarity values: 
Similarity Value: 0.6780517952143867
phytochrome is the red far red absorbing photoreceptor active in photomorphogenesis the apoprotein of which is encoded by a small gene family phya phyb phyc phyd and phye a novel phytochrome b deficient mutant phyb was isolated from a screen of ems mutagenised arabidopsis m seed phyb carries a g to a base substitution at the splice site g nucleotide of intron of phyb the phyb phyb transcript is larger than the wild type phyb transcript and dna sequence analysis showed that the entire intron is retained in the mature phyb transcript of phyb thus the phyb g to a substitution prevents intron splicing the retained intron contains within it an in frame stop codon and the predicted phyb apoprotein thus terminates prematurely phyb is therefore likely to be a null allele of phyb consistent with the observation that the phenotype conferred by phyb is as severe as that conferre